# Model
Use the given preprocessed input `X`, output `y`, evaluation functions `evaluate(y_pred, y_true)`, to train a chosen model (and save the model)  
Resulting model will be class named `Model` with methods:  
* fit(X, y) - train the model
* predict(X) - predict the output for given input
* save_model(path) - save the model to the given path
* load_model(path) - load the model from the given path
* evaluate() - evaluate the model performance

### KNN (K Nearest Neighbors)
https://scikit-learn.org/1.5/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import joblib

class KNN:
    def __init__(self, X, y, evaluate):
        self.evaluate_ = evaluate
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
        self.knn = KNeighborsClassifier(n_neighbors=10)
        
    def fit(self):
        self.knn.fit(self.X_train, self.y_train)

    def predict(self, X):
        return self.knn.predict(X)

    def evaluate(self):
        y_pred = self.predict(self.X_test)
        return self.evaluate_(y_pred, self.y_test)

    def save_model(self, path):
        joblib.dump(self.knn, path)

    def load_model(self, path):
        self.knn = joblib.load(path)

### Logistic Regression

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LR
from sklearn.multioutput import MultiOutputClassifier

class LogisticRegression:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.lr = MultiOutputClassifier(LR(penalty=None))

	def fit(self):
		self.lr.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.lr.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, filename):
		joblib.dump(self.lr, filename)

	def load_model(self, filename):
		self.lr = joblib.load(filename)


### Decision Trees

In [ ]:
from sklearn import tree

class DecisionTree:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.tree = tree.DecisionTreeClassifier()

	def fit(self):
		self.tree.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.tree.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, filename):
		joblib.dump(self.tree, filename)

	def load_model(self, filename):
		self.tree = joblib.load(filename)

### Random Forest

https://scikit-learn.org/1.5/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
from sklearn.ensemble import RandomForestClassifier

class RandomForest:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.rfc = RandomForestClassifier()

	def fit(self):
		self.rfc.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.rfc.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, filename):
		joblib.dump(self.rfc, filename)

	def load_model(self, filename):
		self.rfc = joblib.load(filename)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

class NaiveBayes:
	def __init__(self, X, y, evaluate):
		self.evaluate_ = evaluate
		self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2)
		self.nb = MultiOutputClassifier(MultinomialNB())

	def fit(self):
		self.nb.fit(self.X_train, self.y_train)
		
	def predict(self, X):
		return self.nb.predict(X)
	
	def evaluate(self):
		y_pred = self.predict(self.X_test)
		return self.evaluate_(y_pred, self.y_test)
	
	def save_model(self, filename):
		joblib.dump(self.nb, filename)

	def load_model(self, filename):
		self.nb = joblib.load(filename)